In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [3]:
from kaggle.competitions import twosigmanews
from IPython.core.display import display
env = twosigmanews.make_env()

Loading the data... This could take a minute.
Done!


In [4]:
df = env._var07.set_index(["assetCode", "time"], verify_integrity=True)
df["confidenceValue"] = 2 * (df.returnsOpenNextMktres10 > 0).astype(float) - 1

In [5]:
m = min(df[df.returnsOpenNextMktres10 > 0].returnsOpenNextMktres10.min(), df[df.returnsOpenNextMktres10 < 0].returnsOpenNextMktres10.max() * -1)

In [ ]:
for (
    market_obs_df,
    news_obs_df,
    predictions_template_df,
) in env.get_prediction_days():
    predictions_df = (
        predictions_template_df.set_index("assetCode")
        .join(
            market_obs_df.set_index(["assetCode", "time"])
            .join(df.returnsOpenNextMktres10, how="left")
            .returnsOpenNextMktres10.reset_index()
            .drop("time", axis=1)
            .set_index("assetCode"),
            how="inner",
            lsuffix="_",
        )
        .reset_index()
    )
    predictions_df.confidenceValue = 1 / predictions_df.returnsOpenNextMktres10 * m
    predictions_df.drop("returnsOpenNextMktres10", axis=1)
    
    env.predict(predictions_df)

# env.write_submission_file()